In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# TODO: parametryzacja dla różnych plików/tematów
df = pd.read_csv(".\\data\\E_trajectories\\orb_slam3.csv")

df["Time"] = pd.to_datetime(df["header.stamp.secs"] + df["header.stamp.nsecs"] * 1e-9, unit="s", utc=True)

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS orb_slam3 (
    "header.stamp.secs" BIGINT,
    "header.stamp.nsecs" BIGINT,
    "header.frame_id" TEXT,
    "child_frame_id" TEXT,
    "pose.pose.position.x" DOUBLE PRECISION,
    "pose.pose.position.y" DOUBLE PRECISION,
    "pose.pose.position.z" DOUBLE PRECISION,
    "pose.pose.orientation.x" DOUBLE PRECISION,
    "pose.pose.orientation.y" DOUBLE PRECISION,
    "pose.pose.orientation.z" DOUBLE PRECISION,
    "pose.pose.orientation.w" DOUBLE PRECISION,
    "pose.covariance" TEXT,
    time TIMESTAMPTZ PRIMARY KEY
);
"""
cur.execute(create_table_query)
conn.commit()

cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")
cur.execute("SELECT create_hypertable('orb_slam3', 'time', if_not_exists => TRUE);")
conn.commit()

records = df.to_dict('records')

values = [
    (
        r["header.stamp.secs"],
        r["header.stamp.nsecs"],
        r["header.frame_id"],
        r["child_frame_id"],
        r["pose.pose.position.x"],
        r["pose.pose.position.y"],
        r["pose.pose.position.z"],
        r["pose.pose.orientation.x"],
        r["pose.pose.orientation.y"],
        r["pose.pose.orientation.z"],
        r["pose.pose.orientation.w"],
        r["pose.covariance"],
        r["Time"]
    )
    for r in records
]

insert_query = """
INSERT INTO orb_slam3 (
    "header.stamp.secs",
    "header.stamp.nsecs",
    "header.frame_id",
    "child_frame_id",
    "pose.pose.position.x",
    "pose.pose.position.y",
    "pose.pose.position.z",
    "pose.pose.orientation.x",
    "pose.pose.orientation.y",
    "pose.pose.orientation.z",
    "pose.pose.orientation.w",
    "pose.covariance",
    time
) VALUES %s
ON CONFLICT (time) DO NOTHING;
"""

execute_values(cur, insert_query, values)
conn.commit()

cur.close()
conn.close()


In [16]:
import psycopg2
import pandas as pd

def read_from_timescaledb(conn_params, query):
    conn = None
    try:
        conn = psycopg2.connect(**conn_params)
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print("Błąd podczas odczytu danych:", e)
    finally:
        if conn:
            conn.close()

In [17]:
conn_params = {
    "host": "localhost",
    "dbname": "postgres",
    "user": "postgres",
    "password": "password",
    "port": 5432
}

In [19]:
sql_query = sql_query = """
    SELECT 
        time_bucket('1 minutes', time) AS bucket,
        avg("pose.pose.position.x") AS avg_position_x,
        max("pose.pose.position.y") AS max_position_y
    FROM orb_slam3
    GROUP BY bucket
    ORDER BY bucket;
"""
        
df = read_from_timescaledb(conn_params, sql_query)
if df is not None:
    print(df.head())

                     bucket  avg_position_x  max_position_y
0 2021-07-26 10:25:00+00:00       22.099172       74.685085
1 2021-07-26 10:26:00+00:00       86.804120       71.662923
2 2021-07-26 10:27:00+00:00       16.452041       29.747104
3 2021-07-26 10:28:00+00:00       23.625341        7.128540
4 2021-07-26 10:29:00+00:00       77.438821       42.777516


C:\Users\dawid\AppData\Local\Temp\ipykernel_31352\3092973463.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
